In [15]:
import json
import os
import pickle
from pathlib import Path
from joblib import Parallel, delayed

import geopandas as gpd
import shapely
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import rioxarray

from xrspatial import hillshade
from xrspatial import convolution
from datashader.colors import Set1
from datashader.transfer_functions import shade
from datashader.transfer_functions import stack
from datashader.transfer_functions import dynspread
from datashader.transfer_functions import set_background
from datashader.colors import Elevation

from xrspatial import focal, slope
import seaborn as sns
from tqdm import tqdm
from joblib_progress import joblib_progress
from xrspatial.multispectral import ndvi, savi
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (confusion_matrix, ConfusionMatrixDisplay)
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.model_selection import RandomizedSearchCV as RSCV
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss

In [2]:
# paths
high_high_path = '/home/michael/TreeMortality/data/helena/treatment_polys/code15_n5.gpkg'
high_un_path = '/home/michael/TreeMortality/data/helena/treatment_polys/code12_n5.gpkg'
un_high_path = '/home/michael/TreeMortality/data/helena/treatment_polys/code3_n5.gpkg'
un_un_path = '/home/michael/TreeMortality/data/helena/treatment_polys/code0_n5.gpkg'
poly_paths = [high_high_path, high_un_path, un_high_path, un_un_path]

helena_path = Path.cwd().parent / 'data' / 'helena'
crown_path = helena_path / 'crowns'
crown_path_list = [
    c for c
    in crown_path.iterdir()
    if c.suffix == '.gpkg'
    ]

# open treatment polygons
df = pd.concat([gpd.read_file(p) for p in poly_paths])
df = df.drop('area_', axis=1)

We will first check each crown to see if it falls completely within one of the treatment class areas.  If so it will be appended to a datframe of crowns.


In [3]:
# jobs to run in ||
n_jobs = 23

def is_in_treatment(crown_df, row, buf):
    '''Returns only crowns with buffer completely within polygon'''
    crown_df.loc[
        buf.within(row.geometry),
        'treatment'] = row.attribute
    
    return crown_df[crown_df.treatment >= 0 ]


def label_treatment(f):
    crown_df = gpd.read_file(f)
    
    # get total bounds of tile as polygon
    bounds = crown_df.total_bounds
    bbox = shapely.geometry.box(*bounds)

    # use only treatment geometries which touch the tile
    sub_df = df[df.geometry.intersects(bbox)]
    if len(sub_df) > 0:
        # add treatment column
        crown_df['treatment'] = -99
        #buffer crowns
        buf = crown_df.geometry.buffer(10)
        # label treatments of crowns lying completely within poly
        return Parallel(n_jobs=n_jobs)(
            delayed(is_in_treatment)(crown_df, row, buf)
            for _, row in sub_df.iterrows()
            )
    else:
        # return empty df, but add treatment column first
        cols = list(crown_df.columns) + ['treatment']
        empty_df = pd.DataFrame(columns=cols)
        return [empty_df]


In [5]:
with joblib_progress('', total=len(crown_path_list)):
    results =  Parallel(n_jobs=n_jobs)(delayed(label_treatment)(f) for f in crown_path_list)
    

Output()

In [7]:
# results is a list of lists of dfs, so we must flatten to concat
crown_df = pd.concat([item for sublist in results for item in sublist])

In [ ]:
    
results = [label_treatment(f) for f in tqdm(crown_path_list)]
crown_df = pd.concat(results)

At this point we will also add a unique identifier.  Then save `crowns_df` so in case ware interrupted,  we will be able to resume without running the 5 hour block of code above again.

In [8]:
def make_unique_ID(crowns, utm_zone):
    '''
    returns copy of dataframe with new uniqueID column
    with entries of form 'utm_zone_x_y where x and y 
    are rounded to the nearest meter.
    TODO: make it round to nearest even meter to lower precision
    '''
    crowns['UniqueID'] = crowns.geometry.centroid.apply(
        lambda p: f'{utm_zone}_{p.x:.0f}_{p.y:.0f}')
    
    return crowns

# add unique ID
crown_df_ = make_unique_ID(crown_df, '10N')
crown_df_.head()

,IDdalponte,zmax,zmean,zsd,zskew,zkurt,zentropy,pzabovezmean,pzabove2,zq5,...,p2th,p3th,p4th,p5th,pground,n,area,geometry,treatment,UniqueID
0,2.0,5.29,4.970000,0.549181,-1.123796,2.307702,0.313845,75.000000,100.0,4.3030,...,0.000000,0.000000,0.0,0.0,0.0,4,0.0672,"POLYGON ((496566.730 4511249.660, 496566.620 4...",12,10N_496567_4511250
1,3.0,5.93,5.684000,0.387337,-1.405252,3.124655,0.000000,80.000000,100.0,5.1560,...,20.000000,0.000000,0.0,0.0,0.0,5,0.0760,"POLYGON ((496570.930 4511249.740, 496570.640 4...",12,10N_496571_4511250
2,4.0,9.26,7.120000,2.037727,-0.419361,1.899452,0.602060,50.000000,100.0,4.8005,...,25.000000,0.000000,0.0,0.0,0.0,4,0.0638,"POLYGON ((496589.250 4511249.710, 496589.000 4...",12,10N_496589_4511250
3,10.0,13.39,11.101111,1.725648,0.357462,1.321602,0.435405,44.444444,100.0,9.4440,...,44.444444,0.000000,0.0,0.0,0.0,9,0.1551,"POLYGON ((496743.460 4511249.750, 496743.330 4...",12,10N_496743_4511250
4,11.0,9.49,6.605000,2.987673,-0.013348,1.025824,0.439247,50.000000,100.0,3.6300,...,50.000000,16.666667,0.0,0.0,0.0,6,0.1148,"POLYGON ((496775.400 4511249.590, 496775.330 4...",12,10N_496775_4511250


In [9]:
# save
crown_df_.to_file(helena_path / 'crowns_with_treatment_label.gpkg')

In [11]:
# make sure the number of treatments is reasonable
crown_df_.treatment.value_counts()

treatment
0     2794929
3      601844
12     468077
15      11980
Name: count, dtype: int64

In [12]:
# now finally we kno is safe to make crown_df = crown_df_ 
crown_df = crown_df_

## Topographic Position Index
In order to look at the effects of slope position on tree mortality we will use an algorithm that replicates the Topographic Position Index (TPI). 

$$
\text{TPI} = round((\text{DEM} - \text{focalmean}(\text{DEM}, \;annulus)) + 0.5)
$$
where _annulus_ is an anuulus defined by an inner and outer radius $, r_{inner}$ and $r_{outer}$, 
In keeping consistent with the methods of Kane et al.\cite{kane2015} we will use $r_{outer}$ of 100 m, 250 m, 500 m, 1000 m, and 2000 m outer radii. Since the authors do not specify the inner radius used, here we will use the rule that $r_{inner} = \frac{r_{outer}}{2}$.


In [17]:
# read the DEM
dem = rioxarray.open_rasterio(helena_path / 'helena_DEM.tif')
slope_agg = slope(dem[0])


In [21]:

def topographic_position_index(dem, outer_radius, res=1):
    '''
    returns topological position index
    using r_inner = r_outer /2
    '''
    inner_radius = round(outer_radius / 2)
    kernel = convolution.annulus_kernel(res, res, outer_radius, inner_radius)
    return dem - focal.apply(dem, kernel)

tpi = topographic_position_index(dem[0], 100)
tpi

In [ ]:
tpi_terrain = hillshade(tpi)
tpi_terrain_shaded = shade(
    tpi_terrain, cmap=["white", "black"], alpha=255, how="linear"
)
illuminated = hillshade(dem)
illuminated_shaded = shade(illuminated, cmap=['gray', 'white'], alpha=255, how='linear')
stack(illuminated_shaded, tpi_terrain_shaded)

In [ ]:
def slope_position(tpi, slope):
    std_dev = tpi.std()
    arr = tpi.to_numpy()
    slope_pos = np.zeros_like(arr)
    
    # class description breakpoints
    slope_pos[arr > std_dev] = 1 # ridge
    slope_pos[0.5 * std_dev < arr <= std_dev] = 2 # upper slope
    slope_pos[(-0.5 * std_dev < arr < 0.5 * std_dev) & (slope_agg > 5)] = 3  # middle slope
    slope_pos[(-0.5 * std_dev < arr < 0.5 * std_dev)  & (slope_agg <= 5)] = 4  # flats slope
    slope_pos[-std_dev < arr < -0.5 * std_dev] = 5 # lower slope
    slope_pos[arr < -std_dev] = 6 # valleys
    
    # numpy --> dataArray
    dset = tpi.to_dataset(name='whatever')
    dset['slope_pos'] = (('y', 'x'), slope_pos)
    return dset.slope_pos

slope_pos = slope_position(tpi, slope)
slope_pos

In [ ]:
os.makedirs(helena_path / 'TPI')

def inner_func(slope_pos, row):
    zult = round(np.median(slope_pos.rio.clip([row.geometry]).to_numpy()))
    return (row.UniqueID, zult)

results = []

for r in tqdm([100, 250, 500, 1000, 2000]):
    # add column full of no data vals
    crown_df[f'slope_position_{r}'] = -999
    
    # calc tpi and slope position
    tpi = topographic_position_index(dem, r)
    slope_pos = slope_position(tpi, slope)
    
    # save as tiffs
    tpi.rio.to_raster(helena_path / 'TPI' / f'tpi_{r}.tif')
    slope_pos.rio.to_raster(helena_path / 'TPI' / f'slope_pos_{r}.tif')

    # free a little memory    
    del tpi

    # attach slope positions to crowns
    results.append(
        pd.DataFrame(
            Parallel(n_jobs=n_jobs)(
                delayed(inner_func)(slope_pos, row)
                for _, row in crown_df.iterrows()
            ),
            columns=['UniqueID', f'slope_position_{r}']
        )
    )

# join all results on UniqueID
for data_frame_thing in results:
    crown_df = crown_df.join(
        data_frame_thing,
        on='UniqueID'
    )


In [ ]:
for r in [100, 250, 500, 1000, 2000]:
    ...
    # calculate TPI
    
    # make sample stratified by treatment and TPI class

Now we will create a two-level stratified sample of crowns based on treatment and TPI.

In [ ]:
# dict to hold samples
dict_of_samples = {}

# split into groups based on treatment
for tr in [0, 3, 12, 15]:
    df_x = crown_df[crown_df.treatment == tr]

    # for each treatment, split based on slope position.
    sub_dict = {}
    for pos in [1, 2, 3, 4, 5, 6]:
        sub_dict[f'slope_position_{r}'] = df_x[df_x[f'slope_position_{r}'] == pos]
    dict_of_samples[f'treatment_{tr}'] = sub_dict

# get largest possible even sample for each treatment
for key1, val_dict in dict_of_samples.items():
    # get smallest  TPI population for given treatment
    n = min([len(d) for d in val_dict[key1]])
    # now sample each to that size
    for key2, val in val_dict.items():
        dict_of_samples[key1][key2] = dict_of_samples[key1][key2].sample(n=n)
    

Next we need to engineer the features for our model, as done in `src/mortality_classification_geographic_holdouts.ipynb`